In [33]:
!pip install -q sentence-transformers faiss-cpu transformers gradio

In [34]:
#Importing the required libraries

import faiss
import gradio as gr
from sentence_transformers import SentenceTransformer
from transformers import pipeline

# Loading embedding model
embedder = SentenceTransformer('all-MiniLM-L6-v2')

# Loading local text generation model (bart-large-cnn)
generator = pipeline("text2text-generation", model="facebook/bart-large-cnn", device=0 if __import__('torch').cuda.is_available() else -1)

Device set to use cpu


In [35]:
game_lore = [
    "Dragons once ruled the skies of Tamriel, feared and revered by all.",
    "The Empire was founded by Tiber Septim, a legendary warrior and ruler.",
    "The Thieves Guild operates in the shadows, thriving in major cities.",
    "Magic is regulated by the Mages Guild, which trains all aspiring wizards.",
    "The Dark Brotherhood is a secretive assassin organization bound by a sacred code."
]

# Embedding the lore
lore_embeddings = embedder.encode(game_lore, convert_to_numpy=True)

# Building the FAISS index
index = faiss.IndexFlatL2(lore_embeddings.shape[1])
index.add(lore_embeddings)

In [36]:
def answer_question(question):

    # Embedding the question
    question_embedding = embedder.encode([question], convert_to_numpy=True)

    # Searching most similar lore
    _, indices = index.search(question_embedding, k=1)
    matched_lore = game_lore[indices[0][0]]

    # Building prompt for local model
    prompt = f"Context: {matched_lore}\nQuestion: {question}\nAnswer:"

    # Generating the answer locally
    result = generator(prompt, max_length=100, do_sample=False)[0]['generated_text']

    return result

In [37]:
#Using gradio deploy

gr.Interface(
    fn=answer_question,
    inputs="text",
    outputs="text",
    title="Local Game Lore Assistant",
    description="Ask lore questions and get instant answers with a local Hugging Face model."
).launch(share=True)

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://78a7644a47ba522866.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
